In [2]:
import os
import sys
import shutil
import pdfplumber
import os
import re
# Supprimer complètement les erreurs
sys.stderr = open(os.devnull, 'w')

# === Fonctions Utilitaires ===

def clean_value(value):
    return ''.join(filter(str.isdigit, value)) if value else ""

def extract_data(text, keyword):
    for line in text.split('\n'):
        if keyword in line:
            parts = line.split()
            for i, part in enumerate(parts):
                if keyword in part and i + 1 < len(parts):
                    return parts[i + 1]
    return ""

def extract_lieferschein_from_text(text):
    return clean_value(extract_data(text, "Lieferschein")) if "Lieferschein" in text else ""

def extract_belegnummer_from_text(text):
    return clean_value(extract_data(text, "Beleg-Nr.")) if "Beleg-Nr." in text else ""

# === Chemins ===

lieferschein_folder = r"D:/Work_on_Benz/Lieferung_Benz"
rechnung_folder = r"D:/Work_on_Benz/Rechnung_Benz"
matched_folder = r"D:/Work_on_Benz/Result_folder/matched"
unmatched_folder = r"D:/Work_on_Benz/Result_folder/unmatched"
os.makedirs(os.path.join(unmatched_folder, "Lieferschein"), exist_ok=True)
os.makedirs(os.path.join(unmatched_folder, "Rechnung"), exist_ok=True)
os.makedirs(matched_folder, exist_ok=True)

# === Extraction des fichiers avec leurs valeurs associées ===

lieferschein_map = {}  # {valeur: chemin du fichier}
for f in os.listdir(lieferschein_folder):
    if f.lower().endswith(".pdf"):
        path = os.path.join(lieferschein_folder, f)
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    val = extract_belegnummer_from_text(text)
                    if val:
                        lieferschein_map[val] = path
                        break

rechnung_map = {}  # {valeur: chemin du fichier}
for f in os.listdir(rechnung_folder):
    if f.lower().endswith(".pdf"):
        path = os.path.join(rechnung_folder, f)
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    val = extract_lieferschein_from_text(text)
                    if val:
                        rechnung_map[val] = path
                        break

# === Comparaison et tri ===

matched_keys = set(lieferschein_map.keys()) & set(rechnung_map.keys())

# Fichiers appariés
for key in matched_keys:
    match_dir = os.path.join(matched_folder, key)
    os.makedirs(os.path.join(match_dir, "Lieferschein"), exist_ok=True)
    os.makedirs(os.path.join(match_dir, "Rechnung"), exist_ok=True)
    
    shutil.copy2(lieferschein_map[key], os.path.join(match_dir, "Lieferschein", os.path.basename(lieferschein_map[key])))
    shutil.copy2(rechnung_map[key], os.path.join(match_dir, "Rechnung", os.path.basename(rechnung_map[key])))

# Fichiers non appariés
for key, path in lieferschein_map.items():
    if key not in matched_keys:
        shutil.copy2(path, os.path.join(unmatched_folder, "Lieferschein", os.path.basename(path)))

for key, path in rechnung_map.items():
    if key not in matched_keys:
        shutil.copy2(path, os.path.join(unmatched_folder, "Rechnung", os.path.basename(path)))

print("Classement terminé.")


Classement terminé.


In [3]:
import os
import pdfplumber
import pandas as pd

# === Fonctions Utilitaires ===

def clean_value(value):
    return ''.join(filter(str.isdigit, value)) if value else ""

def extract_data(text, keyword):
    for line in text.split('\n'):
        if keyword in line:
            parts = line.split()
            for i, part in enumerate(parts):
                if keyword in part and i + 1 < len(parts):
                    return parts[i + 1]
    return ""

def extract_lieferschein(text):
    if "Lieferschein" in text:
        return clean_value(extract_data(text, "Lieferschein"))
    return ""

def extract_belegnummer(text):
    if "Beleg-Nr." in text:
        return clean_value(extract_data(text, "Beleg-Nr."))
    return ""

# === Chemins d'entrée ===

lieferschein_folder = r"D:/Work_on_Benz/Lieferung_Benz"
rechnung_folder = r"D:/Work_on_Benz/Rechnung_Benz"

# Listes pour stocker les données extraites
beleg_from_lieferschein = []
lieferschein_from_rechnung = []

# === Parcourir les fichiers de livraison pour extraire Belegnummer ===

for f in os.listdir(lieferschein_folder):
    if f.lower().endswith(".pdf"):
        path = os.path.join(lieferschein_folder, f)
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    value = extract_belegnummer(text)
                    if value:
                        beleg_from_lieferschein.append(value)
                        break
            else:
                beleg_from_lieferschein.append("")  # rien trouvé

# === Parcourir les fichiers de facture pour extraire Lieferschein ===

for f in os.listdir(rechnung_folder):
    if f.lower().endswith(".pdf"):
        path = os.path.join(rechnung_folder, f)
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    value = extract_lieferschein(text)
                    if value:
                        lieferschein_from_rechnung.append(value)
                        break
            else:
                lieferschein_from_rechnung.append("")  # rien trouvé

# === Ajuster les tailles des listes pour création du DataFrame ===

max_len = max(len(beleg_from_lieferschein), len(lieferschein_from_rechnung))
beleg_from_lieferschein.extend([""] * (max_len - len(beleg_from_lieferschein)))
lieferschein_from_rechnung.extend([""] * (max_len - len(lieferschein_from_rechnung)))

# === Création et export du DataFrame ===

df = pd.DataFrame({
    'Belegnummer (dans Lieferscheine)': beleg_from_lieferschein,
    'Lieferschein (dans Rechnungen)': lieferschein_from_rechnung
})

output_excel = r"D:/Work_on_Benz/extracted_swapped_data.xlsx"
df.to_excel(output_excel, index=False)

print(f"Fichier Excel généré avec les données extraites inversées : {output_excel}")


Fichier Excel généré avec les données extraites inversées : D:/Work_on_Benz/extracted_swapped_data.xlsx
